In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn')

In [3]:
fnm1 = pd.read_csv('FNM_MLLD_202109.txt', sep="|")
fnm2 = pd.read_csv('FNM_MLLD_202110.txt', sep="|")

/Users/qimingfeng/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (26,27,29,33,40,43,62,63,66,71,111,112,113,114) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/qimingfeng/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (27,33,40,43,63,66,71,111,112,113,114) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
fnm3 = pd.read_csv('FNM_MLLD_202111.txt', sep="|")
fnm4 = pd.read_csv('FNM_MLLD_202112.txt', sep="|")

/Users/qimingfeng/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (27,33,40,43,63,71,111,112,113,114) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
fnm5 = pd.read_csv('FNM_MLLD_202201.txt', sep="|")
fnm6 = pd.read_csv('FNM_MLLD_202202.txt', sep="|")

In [6]:
def enrich_monthly_principle_payment(df):
    df["Monthly Interest Rate"] = df["Current Net Interest Rate"]/1200
    df["I*( 1 + I )^N"] = df["Monthly Interest Rate"]*(1+df["Monthly Interest Rate"])**df["Loan Term"]
    df["( 1 + I )^N-1"] = (1+df["Monthly Interest Rate"])**(df["Loan Term"])-1
    df["Monthly payment"] = (df["Mortgage Loan Amount"]*df["I*( 1 + I )^N"])/df["( 1 + I )^N-1"]
    df["Monthly interest"] = df["Current Investor Loan UPB"]*df["Monthly Interest Rate"]
    df["Monthly Principal Payment"] = df["Monthly payment"] - df["Monthly interest"]
    return df

In [7]:
def get_cpr(df1,df2, level):
    df1_combo = df1[[level,"Current Investor Loan UPB","Monthly Principal Payment"]]
    df2_combo = df2[[level,"Current Investor Loan UPB","Monthly Principal Payment"]]
    df1_group = df1_combo.groupby(level).sum()
    df2_group = df2_combo.groupby(level).sum()
    df_total = pd.merge(df1_group, df2_group,on=level)
    df_total["Scheduled Principle Balance"] = df_total["Current Investor Loan UPB_x"] - df_total
    ["Monthly Principal Payment_x"]
    df_total["Prepayment"] = df_total["Scheduled Principle Balance"] - df_total["Current Investor Loan UPB_y"]
    df_total["SMM"] = df_total["Prepayment"]/df_total["Scheduled Principle Balance"]
    df_total["CPR"] = ((1-(1-df_total["SMM"])**12))
    df_total =  df_total.reset_index()
    df_total_result = df_total[[level,"CPR"]]
    
    return df_total_result


In [8]:
df1 = fnm1
df1 = enrich_monthly_principle_payment(df1)
df2 = fnm2
df2 = enrich_monthly_principle_payment(df2)
df3 = fnm3
df3 = enrich_monthly_principle_payment(df3)
df4 = fnm4
df4 = enrich_monthly_principle_payment(df4)
df5 = fnm5
df5 = enrich_monthly_principle_payment(df5)
df6 = fnm6
df6 = enrich_monthly_principle_payment(df6)

#df_CUSIP = pd.read_excel("cusip2.xlsx")
#DATA = pd.merge(result, df_CUSIP,on='CUSIP')
#DATA


In [30]:
level = "CUSIP" #can test for other variables
result1 = get_cpr(df1,df2, level)
result1.rename(columns = {'CPR':'CPR_2109-2110'}, inplace = True)

result2 = get_cpr(df2,df3,level)
result2.rename(columns = {'CPR':'CPR_2110-2111'}, inplace = True)

result3 = get_cpr(df3,df4,level)
result3.rename(columns = {'CPR':'CPR_2111-2112'}, inplace = True)

result4 = get_cpr(df4,df5,level)
result4.rename(columns = {'CPR':'CPR_2112-2201'}, inplace = True)

result5 = get_cpr(df5,df6,level)
result5.rename(columns = {'CPR':'CPR_2201-2202'}, inplace = True)

In [31]:
Total_CPR = pd.merge(result1, result2,on=level)
Total_CPR = pd.merge(Total_CPR, result3,on=level)
Total_CPR = pd.merge(Total_CPR, result4,on=level)
Total_CPR = pd.merge(Total_CPR, result5,on=level)
Total_CPR

,CUSIP,CPR_2109-2110,CPR_2110-2111,CPR_2111-2112,CPR_2112-2201,CPR_2201-2202
0,31387LAF8,0.001629,0.001669,0.001708,0.001749,0.001791
1,3138ALP27,-0.001230,-0.001210,-0.001190,-0.001171,-0.001150
2,3138ALP35,0.003212,0.003244,0.003275,0.003307,0.008027
3,3138EDG96,0.355336,0.355772,0.389115,0.487954,0.640316
4,3138EDHB0,0.015302,0.012788,0.006408,0.025439,0.005250
...,...,...,...,...,...,...
135695,31418DZU9,0.010868,0.019693,0.036223,0.044224,0.051704
135696,31418DZV7,0.054237,0.056211,0.074867,0.078586,0.058333
135697,31418DZW5,0.100835,0.123832,0.176241,0.175964,0.139590
135698,31418DZX3,0.191203,0.216896,0.326621,0.383937,0.278658


In [1]:
#pd.set_option('display.max_rows',150)
#df_CUSIP = pd.read_excel("cusip2.xlsx")
#DATA = pd.merge(Total_CPR, df_CUSIP,on=level)
#DATA
#DATA.to_csv("CPR_Calculation_202109-202202.csv", index = False, header=True)